In [ ]:
import json
import pandas as pd
import numpy as np

from scraper import AmazonScraper, OverdriveScraper

In [ ]:
with open('books.json') as books:
    book_json = json.load(books)
    print(book_json.keys())

In [ ]:
pd.json_normalize(book_json)

In [ ]:
# Move wish lists in
reformatted_list = []
for bookshelf, books in book_json.items():
    reformatted_list.append({'bookshelf': bookshelf, 'books': books})

In [ ]:
books_df = pd.json_normalize(reformatted_list, record_path='books', meta='bookshelf')

In [ ]:
books_df.tail()

In [ ]:
books_df.info()

In [ ]:
print(len(books_df))
for column in books_df.columns:
    try:
        nunique = books_df[column].nunique()
        vals=''
        if nunique <=10:
            vals = books_df[column].unique()
        print(column,nunique, vals)
    except:
        print(column, 'skip')


In [ ]:
trimmed_df = books_df.drop([
    'etag',
    'kind',
    'volumeInfo.readingModes.image',
    'volumeInfo.allowAnonLogging',
    'volumeInfo.panelizationSummary.containsEpubBubbles',
    'volumeInfo.panelizationSummary.containsImageBubbles',
    'userInfo.updated',
    'saleInfo.country',
    'saleInfo.offers', 
    'saleInfo.retailPrice.currencyCode', 
    'saleInfo.listPrice.currencyCode',
    'accessInfo.country', 
    'accessInfo.epub.downloadLink', 
    'accessInfo.epub.isAvailable',
    'accessInfo.epub.acsTokenLink',
    'accessInfo.pdf.downloadLink', 
    'accessInfo.pdf.isAvailable',
    'accessInfo.pdf.acsTokenLink',
    'accessInfo.viewability', 
    'accessInfo.embeddable', 
    'accessInfo.textToSpeechPermission',
    'accessInfo.webReaderLink',
    'accessInfo.accessViewStatus',
    'accessInfo.quoteSharingAllowed'], axis=1
)

In [ ]:
trimmed_df = trimmed_df.rename(columns={'id': 'google_id', 
                         'selfLink': 'google_link',
                         'volumeInfo.title': 'title', 
                         'volumeInfo.authors': 'authors',
                         'volumeInfo.publishedDate': 'publishedDate', 
                         'volumeInfo.description': 'description',
                         'volumeInfo.readingModes.text': 'is_text',
                         'volumeInfo.pageCount': 'pageCount', 
                         'volumeInfo.printType': 'printType', 
                         'volumeInfo.categories': 'google_categories',
                         'volumeInfo.averageRating': 'google_avg_rating', 
                         'volumeInfo.ratingsCount': 'google_rating_count',
                         'volumeInfo.maturityRating': 'maturityRating',
                         'volumeInfo.contentVersion': 'google_content_version',
                         'volumeInfo.imageLinks.smallThumbnail': 'smallThumbnail',
                         'volumeInfo.imageLinks.thumbnail': 'thumbnail',
                         'volumeInfo.previewLink': 'google_preview_link',
                         'volumeInfo.infoLink': 'google_info_link',
                         'volumeInfo.canonicalVolumeLink': 'google_canonicalVolumeLink',
                         'volumeInfo.subtitle': 'subtitle',
                         'layerInfo.layers': 'layers',
                         'saleInfo.saleability': 'saleability',
                         'saleInfo.buyLink': 'buyLink',
                         'accessInfo.publicDomain': 'publicDomain',
                         'volumeInfo.publisher': 'publisher',
                         'saleInfo.listPrice.amount': 'google_list_price',
                         'saleInfo.retailPrice.amount': 'google_retail_price',
                         'saleInfo.isEbook': 'is_ebook'
                        }
               )

In [ ]:
trimmed_df.info()

In [ ]:
identifiers = []
for item in trimmed_df['volumeInfo.industryIdentifiers']:
    id_dict = {}
    if type(item) is list:
        for identifier in item:
            id_dict[identifier['type']]=identifier['identifier']
    identifiers.append(id_dict)
id_df = pd.json_normalize(identifiers)
id_df = id_df.fillna('')
id_df=id_df.rename(columns={'OTHER': 'other_identifier'})

In [ ]:
df = pd.concat([trimmed_df, id_df], axis=1)
df= df.drop(['volumeInfo.industryIdentifiers'], axis=1)

In [ ]:
shelf_df = pd.concat([df.google_id, pd.get_dummies(df.bookshelf, prefix='shelf')], axis=1)
shelf_df = shelf_df.pivot_table(index=['google_id'], aggfunc=np.sum, fill_value=-1)

In [ ]:
df = pd.merge(df, shelf_df, on=['google_id']).drop_duplicates(subset='google_id')
df = df.drop('bookshelf', axis=1)

## Clean up types

In [ ]:
cols = {}
for col in ['shelf_2021', 'shelf_2022 Goal Books','shelf_Favorites', 
            'shelf_Have read', 'shelf_Hold', 'shelf_Reading now', 
            'shelf_To read', 'shelf_Wish List']:
    cols[col] = 'bool'
for col in ['pageCount', 'google_rating_count']:
    cols[col] = 'Int64'
for col in ['publishedDate']:
    cols[col] = 'datetime64'
# Objects: 'authors', 'google_categories', 'layers',
df = df.astype(cols)

In [ ]:
df['is_mature'] = (df.maturityRating == 'MATURE')
df = df.drop('maturityRating', axis=1)

In [ ]:
df['google_is_for_sale'] = (df.saleability == 'FOR_SALE')

In [ ]:
df.to_json('transformed_books.json', orient='index')

## Add Amazon data

In [ ]:
books_to_scrape = {book[0]: {'authors': book[2], 'title': book[1]} for book in df[['title', 'authors']].itertuples()}

In [ ]:
scraper = AmazonScraper(cache_file='amazon_data_2.tsv')

In [ ]:
amazon_scraped = await scraper.scrape_books(books_to_scrape)

In [ ]:
amazon_df = pd.DataFrame(amazon_scraped, columns=['index', 'rating', 'link', 'users_rated', 'kindle_price', 'audible_price', 'paperback_price'])
amazon_df = amazon_df.set_index('index')
amazon_df = amazon_df.rename(columns={
        'link':'amazon_link', 
        'users_rated': 'amazon_n_rated', 
        'rating': 'amazon_rating', 
    })

In [ ]:
df = df.join(amazon_df, how='left', rsuffix='amazon')
df.amazon_n_rated = df.amazon_n_rated.fillna(0).astype('int64')

In [ ]:
df.amazon_rating = df.amazon_rating.str.split().str.get(0).astype('float64')

In [ ]:
df['is_on_audible'] = (df.audible_price == 0)

In [ ]:
df.info()

## Add Overdrive Data

In [ ]:
async def join_overdrive_data(df, base_url, suffix, cache_file):
    scraper = OverdriveScraper(base_url, cache_file)
    scraped_data = await scraper.scrape_books(books_to_scrape)
    overdrive_df = pd.DataFrame(scraped_data, columns=['audiobook', 'ebook', 'available', 'link', 'index'])
    overdrive_df = overdrive_df.set_index('index')
    
    overdrive_df = overdrive_df.replace(to_replace=['True', 'False'], value=[True, False])
    overdrive_df = overdrive_df.fillna(value=False)
    overdrive_df = overdrive_df.astype({'audiobook': 'bool', 'ebook': 'bool', 'available': 'bool'})

    overdrive_df = overdrive_df.rename(columns={
        'audiobook':'overdrive_audiobook_'+suffix, 
        'ebook': 'overdrive_ebook_'+suffix, 
        'available': 'available_on_overdrive_'+suffix, 
        'link': 'overdrive_link_'+suffix
    })
    
    return df.join(overdrive_df, how='left', rsuffix=suffix)

In [ ]:
df = await join_overdrive_data(df, 'https://dod.overdrive.com/dod-navy/content/search', 'dod', 'overdrive_data.dod.tsv')

In [ ]:
df = await join_overdrive_data(df, 'http://norfolk-va.overdrive.com/search', 'norfolk', 'overdrive_data.nfk.tsv')

In [ ]:
for col_prefix in ('available_on_overdrive', 'overdrive_ebook', 'overdrive_audiobook'):
    for column in df.columns[df.columns.str.startswith(col_prefix)]:
        df[column] = df[column].fillna(False)

## Completed Dataframe

In [ ]:
df = df.groupby(level=0).last()

In [ ]:
df.info()

In [ ]:
df.loc[df['shelf_Wish List']==True].to_json('wish_list_books.json', orient='index')

In [ ]:
df.to_json('books_df.json', orient='index')

In [ ]:
# df = pandas.read_json('books_df.json', orient='index')